# MNIST

MNIST ("Modified National Institute of Standards and Technology") je základní datový set používaný pro trénink různých systémů zpracování obrazu a strojového učení. 

Je to jedna z nejznámějších a nejčastěji používaných datových sad v oboru počítačového vidění. 

MNIST obsahuje velkou databázi ručně psaných číslic a je často využíván jako vstupní bod pro testování algoritmů strojového učení.

Datový set MNIST se skládá z dvou částí:

- Trénovací sada: Obsahuje 60 000 příkladů. Každý příklad je monochromatický obrázek o velikosti 28x28 pixelů, který reprezentuje jednu ručně psanou číslici od 0 do 9.
- Testovací sada: Obsahuje 10 000 příkladů, které jsou používány pro testování naučených modelů. Tyto příklady mají stejný formát jako ty v trénovací sadě.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Načtení a zobrazení dat

MNIST je velmi známý dataset. Je často přirovnáván k Hello world pro strojové učení.

Proto mnoho frameworku, má připravené funkce na jeho stažení.

In [ ]:
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [ ]:
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

In [ ]:
# Zobrazeni surových dat jednoho čísla
# Čísla jsou uložena v matici 28x28 pixelů 
X_train[0]

In [ ]:
# Hodnoty jsou odstíny šedi 0 - 255
# 0 je černá, 255 je bílá
img = X_train[0]
print (f"min:{np.amin(X_train[0])} max:{np.amax(X_train[0])}")

In [ ]:
# funkce pro zobrazení n prvních čísel včetně jejich popisků
def show_images (images, labels, rows=6, cols=10):
    fig, axes = plt.subplots(rows, cols, figsize=(cols, rows))
    for idx in range (0, rows * cols):
        ridx=idx // cols
        cidx=idx % cols
        ax= axes[ridx, cidx]
        ax.axis("off")
        ax.imshow(images[idx], cmap="gray_r")
        ax.set_title(f"{labels[idx]}")
    plt.show()    

In [ ]:
show_images(X_train, Y_train, 2, 10)

# 2. Příprava dat

Standardizace hodnot z rozsahu 0 - 255 na rozsah 0 - 1

In [ ]:
X_train = X_train.astype("float32")/255
X_test = X_test.astype("float32")/255

In [ ]:
# Vstupem do neuronové sítě je vektor. 
# Změníme tvar pole jedné číslice z 28x28 na 784x1
print (X_train.shape)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))
print (X_train.shape)

In [ ]:
X_train[0]

In [ ]:
# pro neuronovou síť potřebujeme upravit i výsledky pomocí binárního encodování
# tentokát použijeme vestavěnou funkci ve frameworku keras
from keras.utils import to_categorical 
Y_train = to_categorical(Y_train, num_classes=10)
Y_test = to_categorical(Y_test, num_classes=10)

In [ ]:
# reprezentace číslice 5
Y_train[0]

# 3. Neuronová síť

* Sekvenční model - data procházejí ze vstupní vrstvy přes skryté vrstvy do výstupní vrstvy.
* Dense vrstva propojuje každý neuron z předchozí vrstvy s každým neuronem ve vrstvě.
* Používají se aktivační funkce RELU a Softmax. 
* Poslední funkce Softmax je použita ve výstupní vrstvě, protože tento projekt je klasifikační.

In [ ]:
import keras
from keras.layers import Dense, Activation, Input
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Input(shape=(784,))) 
model.add(Dense(128, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(10, activation = 'softmax'))

* Kompilace je posledním krokem při vytváření modelu.
* Optimalizátor, který používáme, je Adam. Adam je optimalizační algoritmus, který lze použít místo klasického postupu stochastického gradientního sestupu k iterativní aktualizaci vah sítě na základě trénovacích dat.
* Nákladová funkce - categorical_crossentropy, speciální ztrátová funkce pro klasifikaci (https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class).
* Během učení se počítá další metrika - Accuracy

In [ ]:
model.compile(loss = "categorical_crossentropy",
                optimizer = "adam",
                metrics = ["accuracy"]
                )

In [ ]:
model.summary()

In [ ]:
from keras_visualizer import visualizer 
visualizer(model, file_format='png', view=True)

# 4. Učení

- Maximální počet epoch je 200
- Velikost dávky je 75
- Učení ukončíme, ve chvíly, kdy se přesnost modelu nebude zvyšovat.

In [ ]:
# zastavení učení, když se val_loss nebude příliš měnit
early_stop = keras.callbacks.EarlyStopping(monitor='accuracy', patience=30)

# maximální počet učebních cyklů 500, velikost batch je 5
history=model.fit(X_train, Y_train, epochs=200, batch_size=75, callbacks=[early_stop])

In [ ]:
model.save('classification_model_mnist.keras')

# 5. Historie učení modelu

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Počet epoch')
plt.show()

# 6. Ověření modelu

In [ ]:
# Výpočet predikce testovacích dat
Y_pred = model.predict(X_test)

In [ ]:
Y_pred[0]

In [ ]:
Y_test[0]

In [ ]:
# Klasifikační neuronová síť vrací vektor pravděpodobností příslušnosti do jednotlivých tříd.
# Pokud chceme jako odpověď jednu třídu, většinou použijeme funkci max
import numpy as np
Y_pred_best_answer = np.argmax(Y_pred, axis=-1)
Y_pred_best_answer

In [ ]:
# totéž provedeme s testovacími odpovědmi (realita)
# ideálně se obě pole rovnají
Y_test_best_answer=np.argmax(Y_test, axis=-1)
Y_test_best_answer

In [ ]:
# Vykreslení confusin matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cf_matrix=confusion_matrix(Y_test_best_answer, Y_pred_best_answer)
sns.heatmap(cf_matrix, annot=True)

In [ ]:
# R2 skóre pro nejlepší odpovědi
from sklearn.metrics import r2_score
r2 = r2_score(Y_test_best_answer, Y_pred_best_answer)
print('R2 score: {}'.format(r2))

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print (f"Loss function (categorical_crossentropy): {scores[0]}")
print (f"Accuracy: {scores[1]}")

In [ ]:
def show_wrong_predictions(X_test, Y_test, Y_pred, rows=5, cols=5):    
    """ 
    Zobrazení špatných předpovědí
    První číslice je správná, druhá odhadovaná
    """
    idx = 0
    max_examples = rows * cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols, rows))    
    for i in range(Y_test.shape[0]):    
        if (Y_test[i] != Y_pred[i]):                                    
            ridx=idx // cols
            cidx=idx % cols            
            ax = axes[ridx, cidx]
            ax.axis("off")
            ax.imshow(X_test[i].reshape(28,28), cmap="gray_r")
            ax.set_title(f"{Y_test[i]} != {Y_pred[i]}")            
            idx +=1
            if (idx == max_examples):
                break

In [ ]:
show_wrong_predictions(X_test, Y_test_best_answer, Y_pred_best_answer, 2, 10)

# 7. Použití modelu
Ve chvíli, kdy máme vytrénovaný model, můžeme mu předložit libovolnou matici o rozměrech 28x28

Současná architektura neuronové sítě, ji neumožňuje odpovědět, že "neví" co je na obrázku.

Vždy bude vrace příslušnost do jedné z 10 tříd.

In [ ]:
# náhodně vygenerovaná matice
img = np.random.rand(1, 784) * 2 -1
plt.imshow(img.reshape(28,28), cmap="gray_r")

In [ ]:
img_reshaped = img.reshape(1, 784)
test_predictions = model.predict([img_reshaped])

In [ ]:
for i in range(10):
    print (f"{i}: {test_predictions[0, i]:.5f}")